In [1]:
import requests
url = "http://export.arxiv.org/api/query?search_query=all:deep+learning&start=0&max_results=5"
response = requests.get(url)

In [3]:
import xml.etree.ElementTree as ET

# Parse the XML response
root = ET.fromstring(response.text)

# Extract titles and abstracts
abstracts = []  # List to store abstracts
titles = []  # List to store titles

for entry in root.findall("{http://www.w3.org/2005/Atom}entry"):
    title = entry.find("{http://www.w3.org/2005/Atom}title").text.strip()
    abstract = entry.find("{http://www.w3.org/2005/Atom}summary").text.strip()

    titles.append(title)
    abstracts.append(abstract)

In [4]:
from transformers import pipeline
# Initialize summarization pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

/Users/nitin/Documents/AI Agent project/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Device set to use mps:0


In [5]:
# Summarize abstracts
summaries = [summarizer(abs, max_length=50, min_length=25, truncation=True)[0]["summary_text"] for abs in abstracts]

# Print results
for title, summary in zip(titles, summaries):
    print(f"Title: {title}\nSummary: {summary}\n" + "-"*80)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Title: Opening the black box of deep learning
Summary: The great success of deep learning shows that its technology contains aprofound truth. Understanding its internal mechanism not only has important implications for the development of its technology and effective application in various fields. At present, most of the theoretical research on
--------------------------------------------------------------------------------
Title: Concept-Oriented Deep Learning
Summary: Concepts are the foundation of human deep learning, understanding, and knowledge integration and transfer. We propose concept-oriented deep learning(CODL) which extends (machine) deep learning with concept representations. CODL addresses some
--------------------------------------------------------------------------------
Title: Deep learning research landscape & roadmap in a nutshell: past, present
  and future -- Towards deep cortical learning
Summary: The past, present and future of deep learning is presented in this 

In [6]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
# Initialize HuggingFace Embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create FAISS Vector Store
vector_store = FAISS.from_texts(abstracts, embedding_model)

# Perform a similarity search (RAG Retrieval Step)
query = "best deep learning advancements"
top_docs = vector_store.similarity_search(query, k=2)

# Display Retrieved Documents
for i, doc in enumerate(top_docs):
    print(f"Top {i+1} Result:\n{doc.page_content}\n" + "-"*80)

/var/folders/cv/01jc2kcx1455j48m7hhwcvqw0000gn/T/ipykernel_17287/3038899353.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


Top 1 Result:
We are in the dawn of deep learning explosion for smartphones. To bridge the
gap between research and practice, we present the first empirical study on
16,500 the most popular Android apps, demystifying how smartphone apps exploit
deep learning in the wild. To this end, we build a new static tool that
dissects apps and analyzes their deep learning functions. Our study answers
threefold questions: what are the early adopter apps of deep learning, what do
they use deep learning for, and how do their deep learning models look like.
Our study has strong implications for app developers, smartphone vendors, and
deep learning R\&D. On one hand, our findings paint a promising picture of deep
learning for smartphones, showing the prosperity of mobile deep learning
frameworks as well as the prosperity of apps building their cores atop deep
learning. On the other hand, our findings urge optimizations on deep learning
models deployed on smartphones, the protection of these models, an

In [7]:
# Prerequisites: pip install langchain faiss-cpu transformers requests
import requests
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import pipeline

# Step 1: Define the Agent class (our Research Buddy)
class ResearchBuddyAgent:
    def __init__(self):
        # Load embeddings for RAG
        self.embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
        # Load summarizer (small model for speed)
        self.summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
        self.papers = []  # Store fetched papers

    # Step 2: Fetch papers (simple arXiv API call)
    def fetch_papers(self, topic):
        url = f"http://export.arxiv.org/api/query?search_query=all:{topic}&start=0&max_results=5"
        response = requests.get(url)
        # Parse the XML response
        root = ET.fromstring(response.text)

        # Extract titles and abstracts
        abstracts = []  # List to store abstracts
        titles = []  # List to store titles

        for entry in root.findall("{http://www.w3.org/2005/Atom}entry"):
            title = entry.find("{http://www.w3.org/2005/Atom}title").text.strip()
            abstract = entry.find("{http://www.w3.org/2005/Atom}summary").text.strip()

            titles.append(title)
            abstracts.append(abstract)

    # Step 3: Build RAG vector store and retrieve
    def retrieve_relevant(self, query):
        # Create FAISS vector store from papers
        vector_store = FAISS.from_texts(abstracts, embedding_model)
        # Search for top matches
        top_docs = vector_store.similarity_search(query, k=2)
        return [doc.page_content for doc in top_docs]  # Extract text

    # Step 4: Summarize papers
    def summarize_papers(self, papers):
        summaries = []
        for paper in papers:
            summary = self.summarizer(paper, max_length=30, min_length=15)[0]["summary_text"]
            summaries.append(summary)
        return summaries

    # Step 5: Decide the "best" (simple rule: shortest summary wins)
    def pick_best(self, summaries):
        best = min(summaries, key=len)  # Lazy rule for demo
        return best

    # Step 6: Main agent flow (plan and execute)
    def run(self, topic, query):
        print("Agent: Time to flex my research muscles!")
        # Plan: Fetch → Retrieve → Summarize → Pick
        self.fetch_papers(topic)
        relevant_papers = self.retrieve_relevant(query)
        summaries = self.summarize_papers(relevant_papers)
        best_summary = self.pick_best(summaries)
        print(f"Top papers: {summaries}")
        print(f"Best pick: {best_summary}")

# Run the agent
if __name__ == "__main__":
    agent = ResearchBuddyAgent()
    agent.run(topic="deep learning", query="best deep learning advancements")

Device set to use mps:0


Agent: Time to flex my research muscles!
Top papers: ['First empirical study on 16,500 popular Android apps. Demystifies how smartphone apps exploit deep learning in the wild. Findings', 'The past, present and future of deep learning is presented in this work. We predict that deep cortical learning will be the convergence of deep']
Best pick: First empirical study on 16,500 popular Android apps. Demystifies how smartphone apps exploit deep learning in the wild. Findings
